In [2]:
import torch
import pykeen
from pykeen import predict
from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset

In [3]:
mps_device = torch.device("mps")

In [4]:
ctd_only_model = torch.load('results/baseline/transe_hetionet_only_ctd_relation/trained_model.pkl', mps_device)

In [5]:
dataset = get_dataset(dataset="hetionet")

In [18]:
triples_factory = dataset.training
test_triples = dataset.testing.mapped_triples
ctd_id = torch.as_tensor(triples_factory.relations_to_ids(["CtD"]))
ctd_triples = test_triples[test_triples[:, 1] == ctd_id]

In [36]:
ctd_triples

tensor([[13278,     8, 14823],
        [13931,     8, 14769],
        [13675,     8, 14802],
        [14458,     8, 14788],
        [13982,     8, 14776],
        [14061,     8, 14787],
        [13468,     8, 14723],
        [13795,     8, 14784],
        [13489,     8, 14799],
        [14222,     8, 14769],
        [14340,     8, 14754],
        [13670,     8, 14831],
        [14209,     8, 14809],
        [14115,     8, 14778],
        [13654,     8, 14788],
        [14340,     8, 14745],
        [14509,     8, 14719],
        [13538,     8, 14823],
        [13909,     8, 14723],
        [13214,     8, 14733],
        [13796,     8, 14747],
        [13228,     8, 14831],
        [13581,     8, 14768],
        [14097,     8, 14775],
        [14194,     8, 14788],
        [13561,     8, 14769],
        [13600,     8, 14826],
        [13655,     8, 14799],
        [13625,     8, 14723],
        [13774,     8, 14791],
        [13805,     8, 14805],
        [14005,     8, 14791],
        

In [89]:
def evaluate_semantic_awareness(triples, model, triples_factory):
   final_semantic_awareness_score = 0
   for triple in triples:
        prediction_for_tail = predict.predict_target(model=model, relation="CtD", head=triple[0].item(), triples_factory=triples_factory)
        prediction_for_tail_top_10 = prediction_for_tail.df.nlargest(n=10, columns="score")
        prediction_for_head = predict.predict_target(model=model, relation="CtD", tail=triple[2].item(), triples_factory=triples_factory)
        prediction_for_head_top_10 = prediction_for_head.df.nlargest(n=10, columns="score")
        count_of_expected_head = len(prediction_for_head_top_10[prediction_for_head_top_10["head_label"].str.startswith("Compound")])
        count_of_expected_tail = len(prediction_for_tail_top_10[prediction_for_tail_top_10["tail_label"].str.startswith("Disease")])
        semantic_score = ((count_of_expected_head + count_of_expected_tail) / (len(prediction_for_tail_top_10) + len(prediction_for_head_top_10)))
        final_semantic_awareness_score += semantic_score
   final_semantic_awareness_score = final_semantic_awareness_score / len(triples)
   return final_semantic_awareness_score

In [92]:
sem_score_std_only = evaluate_semantic_awareness(ctd_triples, ctd_only_model, dataset.training)
sem_score_std_only

0.897887323943661

In [93]:
best_hpo2_model = torch.load('results/baseline/transe_hetionet_only_ctd_relation/trained_model.pkl', mps_device)

In [94]:
sem_score_best_hpo2 = evaluate_semantic_awareness(ctd_triples, best_hpo2_model, dataset.training)
sem_score_best_hpo2

0.897887323943661